In [ ]:
from kafka import KafkaConsumer
from kafka import TopicPartition
from datetime import datetime
import pandas as pd
from cassandra.cluster import Cluster
import json
import boto3
from botocore.vendored import requests
from elasticsearch import Elasticsearch
import findspark
findspark.init()
import pyspark.sql 
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
import pandas as pd
from cassandra.cluster import Cluster
import numpy as np
from pyspark.sql.types import *
#------------------------------- SPARK CONTEXT -------------------------------
sc = SparkContext('local')
spark = SparkSession(sc)
sqlContext = SQLContext(sc) 
spark = SparkSession.builder.appName('ml-bank').getOrCreate()

myschema = StructType([ StructField("txnid",StringType(), True)\
                     ,StructField("age", IntegerType(), True)\
                     ,StructField("amount", FloatType(), True)\
                     ,StructField("cc_no", LongType(), True)\
                     ,StructField("cc_provider", IntegerType(), True)\
                     ,StructField("country_code", StringType(), True)\
                     ,StructField("country_index", IntegerType(), True)\
                     ,StructField("country_name", StringType(), True)\
                     ,StructField("day", IntegerType(), True)\
                     ,StructField("gender", StringType(), True)\
                     ,StructField("gender_index", IntegerType(), True)\
                     ,StructField("hour", IntegerType(), True)\
                     ,StructField("loan", StringType(), True)\
                     ,StructField("loan_index", IntegerType(), True)\
                     ,StructField("marital_status", StringType(), True)\
                     ,StructField("marital_status_index", IntegerType(), True)\
                     ,StructField("merchant", StringType(), True)\
                     ,StructField("merchant_index", IntegerType(), True)\
                     ,StructField("minute", IntegerType(), True)\
                     ,StructField("month", IntegerType(), True)\
                     ,StructField("status", StringType(), True)\
                     ,StructField("status_index", IntegerType(), True)\
                     ,StructField("txn_time", StringType(), True)\
                     ,StructField("year", IntegerType(), True)\
                    ])

#------------------------------- Load Model from File System -------------------------------

def MachineLearningModelLoading():
	path="/Users/maitreytalware/Desktop/Kakfa python-mayur-/ML/mlmodel12"
	newmodel=PipelineModel.load(path)
	return newmodel

newmodel=MachineLearningModelLoading()

#------------------------------------ Data Cleaning -------------------------------------

def predicting_status(data_all,newmodel):
	#print("Number of rows: {},  Number of coulumns : {}".format(data_all.count(), len(data_all.columns)))

	#Replace "-" with "_" in column names
	columns_new = [col.replace("-", "_") for col in data_all.columns]
	data_all = data_all.toDF(*columns_new)

	#Print Schema and show top 5 row
	#data_all.printSchema()
	drop_list = ['txnid','merchant','country_name','country_code','marital_status','gender','loan','status',
	'txn_time',
	'year',
	'month',
	'day',
	'hour',
	'minute',
	'country_index','merchant_index','status_index']
	data_all = data_all.select([column for column in data_all.columns if column not in drop_list])	
	#data_all.printSchema()

	#label = "status_index"
	xvars = ["cc_no", "amount",'cc_provider','age','marital_status_index','gender_index','loan_index'] #all numeric
	#print("label = {}".format(label))
	#print("features = {}".format(xvars))
	select_cols = xvars
	#select_cols.append(label)
	data = data_all.select(select_cols)
	pred1 = newmodel.transform(data)
    
	return pred1.select("prediction")



#------------------------------------------- CASSANDRA CONNECTION -------------------------------------------------------
cluster = Cluster()
session = cluster.connect()
session.execute("CREATE KEYSPACE IF NOT EXISTS bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3}")
session.set_keyspace("bigdata")
#session.execute("DROP TABLE testing")
session.execute("CREATE TABLE IF NOT EXISTS testing (txn_id varchar PRIMARY KEY,cc_no bigint ,txn_time varchar, year smallint, month smallint, day smallint, hour smallint, minute smallint,amount double, cc_provider int, merchant varchar,merchant_index smallint, country_name varchar, country_code varchar,country_index smallint, age int, marital_status varchar,marital_status_index smallint,gender varchar,gender_index smallint,loan varchar,loan_index smallint, status varchar,status_index smallint)")
# session.set_keyspace("mykeyspace")
# session.execute("CREATE TABLE IF NOT EXISTS training (txn_id varchar PRIMARY KEY,cc_no bigint , cc_provider int, year smallint, month smallint, day smallint, hour smallint, min smallint,txn_time varchar, merchant varchar, location varchar, country varchar, amount float, status varchar)")

#--------------------------------------------- KAFKA CONSUMER-----------------------------------------------------------
consumer = KafkaConsumer('fifth_topic',group_id='my-tenth-application',bootstrap_servers=['localhost:9092'])
# consumer.assign([TopicPartition('')])
for msg in consumer:
    
	msg=msg.value.decode("utf-8")
	df = spark.read.json(sc.parallelize([msg]))
	df = df.withColumn("age", df["age"].cast(IntegerType()))
	df = df.withColumn("cc_provider", df["cc_provider"].cast(IntegerType()))
	df = df.withColumn("cc_no", df["cc_no"].cast(LongType()))
    
    
	msg=json.loads(msg)

	status_predicted=predicting_status(df,newmodel)
	if status_predicted.collect()[0][0]==0:
		status="Approved"
	else:
		status="Rejected"

	msg["status"]=status
	elastic_data={
	"cc_no":msg["cc_no"],
	"txn_time":msg["txn_time"],
	"year":msg["year"],
	"month":msg["month"],
	"cc_provider":msg["cc_provider"],
	"merchant":msg["merchant"],
	"country_code":msg["country_code"],
	"amount":msg["amount"],
	"age":msg["age"],
	"marital_status":msg["marital_status"],
	"gender":msg["gender"],
	"loan":msg["loan"],
	"status":msg["status"]
	}
	elastic_data = json.dumps(elastic_data)
	#------------------------------------------------- PUTTING INTO CASSANDRA -----------------------------------------------
	strq=" INSERT INTO testing JSON "+"'"+json.dumps(msg)+"'"
	session.execute(strq)
	
# 	#---------------------------------------------------- ELASTIC SEARCH ----------------------------------------------------

	headers={'Content-type':'application/json'}
	API_ENDPOINT="http://localhost:9200/credit/credits/"+msg["txn_id"]
	#print(API_ENDPOINT)
	m=requests.post(url = API_ENDPOINT, data = elastic_data, headers=headers)
consumer.close()